
# Checking the dimensions

In [61]:
import numpy as np

In [62]:
# ptbxl 248
dset_ptbxl = np.load('val_ptbxl_248.npy')
dset_ptbxl.shape

(8772, 12, 248)

In [63]:
# MuJoCO
dset_mjc = np.load('test_mujoco.npy')
dset_mjc.shape

(2000, 100, 14)

In [64]:
#Electricity
dset_elec_train = np.load('train_electricity.npy')
dset_elec_test = np.load('test_electricity.npy')
dset_elec_test.shape

(10, 291, 100, 37)

In [65]:
import einops
dset_elec_train = einops.rearrange(dset_elec_train, 'c1 d seq c2 -> d seq (c1 c2)')
dset_elec_test = einops.rearrange(dset_elec_test, 'c1 d seq c2 -> d seq (c1 c2)')

dset_elec_train.shape

(817, 100, 370)

In [66]:
np.save('train_electricity_v1.npy', dset_elec_train)
np.save('test_electricity_v1.npy' , dset_elec_test)

In [67]:
# ptbxl 1000
dset_ptbxl = np.load('test_ptbxl_1000.npy')
dset_ptbxl.shape

(2203, 12, 1000)

In [68]:
# EETm1
dset_eetm1 = np.load('test_ettm1_1056.npy')
dset_eetm1.shape

(33217, 1056, 7)

# dload yfinance content

In [69]:
import yfinance as yf
import yfinance
from pytickersymbols import PyTickerSymbols
import pandas as pd
end_date="2023-01-10"

In [70]:
stock_data = PyTickerSymbols()
dow30_yahoo = stock_data.get_dow_jones_nyc_yahoo_tickers()
euro50_yahoo = stock_data.get_euro_stoxx_50_frankfurt_yahoo_tickers()

hang = [ "2628.HK","2319.HK","2688.HK","0992.HK"
        ,"9988.HK","0101.HK","1810.HK","9618.HK",
        "1044.HK","2331.HK","0883.HK","0017.HK",
        "1398.HK","0002.HK","9633.HK","2269.HK",
        "0267.HK","0003.HK","1093.HK","1038.HK",
        "0027.HK","6690.HK","0241.HK","6098.HK",
        "1109.HK","0012.HK","2020.HK","3690.HK","1209.HK","0669.HK"]


In [91]:
# Downloading data for all stocks
stock_data_tz_ignored = yf.download( tickers = " ".join(dow30_yahoo+euro50_yahoo+hang) ,  # list of tickers
            # period = "10y",         # time period
            end=end_date,
            interval = "1d",       # trading interval
            ignore_tz = False,      # ignore timezone when aligning data from different exchanges?
            prepost = False,
            auto_adjust= True, # adjust for stock splits etc,
            keepna=True,
            )    
stock_data_tz_ignored.columns =  stock_data_tz_ignored.columns.swaplevel(0, 1)

stock_data = yf.download( tickers = " ".join(dow30_yahoo+euro50_yahoo+hang) ,  # list of tickers
            # period = "10y",         # time period
            end=end_date,
            interval = "1d",       # trading interval
            ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
            prepost = False,
            auto_adjust= True, # adjust for stock splits etc,
            keepna=True,
            
            )    
stock_data.columns =  stock_data.columns.swaplevel(0, 1)

[*********************100%***********************]  160 of 160 completed

4 Failed downloads:
- 0VD.F: 1d data not available for startTime=-2208994789 and endTime=1673305200. Only 100 years worth of day granularity data are allowed to be fetched per request.
- STLA.F: No timezone found, symbol may be delisted
- NL0013654783.F: No timezone found, symbol may be delisted
- NL00150001Q9.F: No timezone found, symbol may be delisted
[*********************100%***********************]  160 of 160 completed

4 Failed downloads:
- 0VD.F: 1d data not available for startTime=-2208994789 and endTime=1673305200. Only 100 years worth of day granularity data are allowed to be fetched per request.
- STLA.F: No timezone found, symbol may be delisted
- NL0013654783.F: No timezone found, symbol may be delisted
- NL00150001Q9.F: No timezone found, symbol may be delisted


In [92]:
# Removing data with less than 10 years of records

def filter_data_by_cutoff_date( df_stock:pd.DataFrame, timedelta:str ):

    
    latest_date = df_stock.index[-1]
    cutoff_date = latest_date - pd.Timedelta(timedelta)

    # Gathering the rows which have no data prior to cutoff date
    df_stock_before_certain_date = df_stock.loc[ df_stock.index < cutoff_date  ]

    nan_values = df_stock_before_certain_date.isna()
    mask = nan_values.all(axis=0)

    df_stock = df_stock.drop( df_stock.columns[mask], axis=1) 
    
    return df_stock

timedelta ='10Y'
stock_data_tz_ignored = filter_data_by_cutoff_date(stock_data_tz_ignored, timedelta  )
stock_data = filter_data_by_cutoff_date(stock_data, timedelta  )
        

In [110]:
# Getting data for the currency values of Euro/USD and CNY/USD
eurusd_data = yf.download( 'EURUSD=X',end=end_date )
cnyusd_data = yf.download( 'CNYUSD=X',end=end_date )

# Finding latest Date at which all currency date is available
eurusd_d0 = eurusd_data.index[0]
cnyusd_d0 = cnyusd_data.index[0]

latest_date_currency = max(eurusd_d0, eurusd_d0 )



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [147]:
# Only Keep Stock & Currency Data after date for which they are ALL available
eurusd_data = eurusd_data[eurusd_data.index> latest_date_currency ]
cnyusd_data = cnyusd_data[cnyusd_data.index> latest_date_currency ]
stock_data = stock_data[stock_data.index> latest_date_currency ]
stock_data_tz_ignored = stock_data_tz_ignored[ stock_data_tz_ignored.index.map( lambda x:  x.tz_localize(None) ) > latest_date_currency ]
# NOTE: questionable correctness since tz is assumed to be based on NY time but tz_ignored is general


array([ True,  True,  True, ...,  True,  True,  True])

In [45]:
# Saving to pickle
import pickle 

pickle.dump( stock_data_tz_ignored, open('train_stock.pkl','wb'))
pickle.dump( stock_data, open('train_stock.pkl','wb'))


In [ ]:
# Download Data All together



In [ ]:
stock.info